In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import dask

from dask.distributed import Client
from datetime import datetime

client = Client(n_workers=8)

# CADES configurationt
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(project='birthright', queue='high_mem_cd', cores=30, memory='150 GB', job_extra=["-N 1 -t 4:00:00"], interface="ib0")
# client = Client(cluster)

In [2]:
counties = dd.read_csv("data/mapbox/production_v02/county_daily_24_0/US/**/*.csv", sep='|')
counties.head()

,geography,agg_day_period,activity_index_total
0,USA248021,2020-03-01,0.017817
1,USA224005,2020-03-01,0.089171
2,USA237035,2020-03-01,0.027586
3,USA205045,2020-03-01,0.030865
4,USA220149,2020-03-01,0.005965


In [3]:
def split_geography(row):
    geog = row.geography
    row['state_fips'] = geog[4:6]
    row['county_fips'] = geog[-3:]
    return row

counties = counties.apply(split_geography, axis=1, meta={'geography': 'str', 'agg_day_period': 'object', 'activity_index_total': 'float64', 'state_fips': 'str', 'county_fips': 'str'})

In [4]:
state_fips = dd.read_csv("data/reference/state_fips.csv", dtype={'state_fips': 'str'}).set_index('state_fips')

In [5]:
counties = counties.join(state_fips[['state_name']], on='state_fips')

In [6]:
counties.groupby(['state_name', 'agg_day_period']).mean().reset_index().to_csv("data/output/Mabbox-State-Mobility.csv")

['/Users/raac/Development/covid/mobility-analysis/data/output/Mabbox-State-Mobility.csv/0.part']

In [167]:
c2 = counties.copy()

In [169]:
from scipy.stats import linregress
from datetime import datetime, timedelta

def compute_regression(values):
    y = np.array(values)
    x = np.arange(len(values))
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return slope

def filter_dates(row):
    return  #.apply(np.array).apply(compute_regression)

def print_it(vals):
    print(len(vals))
    return vals.sum()

c2['date'] = c2['agg_day_period'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date(), meta=('date', 'object'))
c2 = c2.drop(['agg_day_period'], axis=1)
d = datetime.today() - timedelta(days=3)

c2 = c2[c2.date >= d]
# c2.groupby(['geography']).apply(lambda x: x.activity_index_total.transform(print_it), meta=('trend', 'object')).reset_index().head()
c2.groupby(['geography']).activity_index_total.sum().head()

# c2.groupby(['geography']).apply(lambda x: x.activity_index_total.rolling(7).sum()).reset_index().head()
# c2['hello'] = c2.groupby(['geography']).activity_index_total.transform(lambda x: 1).head() #.rolling(7).apply(lambda x: len(x)).head()
# s = c2.groupby(['geography']).activity_index_total.sum()
#ID = AID.set_index('AID').drop('ANumOfF', axis=1).join(s).reset_index()
# c2['summed'] = c2.geography.map(s)
# c2.head()

ValueError: Metadata inference failed in `ge`.

Original error is below:
------------------------
TypeError("'>=' not supported between instances of 'str' and 'datetime.datetime'")

Traceback:
---------
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/dask/dataframe/utils.py", line 172, in raise_on_meta_error
    yield
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/dask/dataframe/core.py", line 4660, in elemwise
    meta = partial_by_order(*parts, function=op, other=other)
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/dask/utils.py", line 1089, in partial_by_order
    return function(*args2, **kwargs)
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/pandas/core/ops/common.py", line 64, in new_method
    return method(self, other)
  File "/Users/raac/Development/covid/mobility-analysis/venv/lib/python3.7/site-packages/pandas

In [50]:
c2.head()

,geography,agg_day_period,activity_index_total,state_fips,county_fips,state_name
0,USA248021,2020-03-01,0.017817,48,021,Texas
1,USA224005,2020-03-01,0.089171,24,005,Maryland
2,USA237035,2020-03-01,0.027586,37,035,North Carolina
3,USA205045,2020-03-01,0.030865,05,045,Arkansas
4,USA220149,2020-03-01,0.005965,20,149,Kansas
